# Publications markdown generator for academicpages

Takes a set of bibtex of publications and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). 

The core python code is also in `pubsFromBibs.py`. 
Run either from the `markdown_generator` folder after replacing updating the publist dictionary with:
* bib file names
* specific venue keys based on your bib file preferences
* any specific pre-text for specific files
* Collection Name (future feature)

TODO: Make this work with other databases of citations, 
TODO: Merge this with the existing TSV parsing solution

TODO: make it parse from bib with entries not necessarily all written by author

In [1]:
from pybtex.database.input import bibtex
import pybtex.database.input.bibtex
import pybtex
import time
import html
import os
import re

In [2]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [7]:
APA.format_doi??

Signature: APA.format_doi(e)
Docstring: <no docstring>
Source:   
    def format_doi(self, e):
        # Based on urlbst format.doi
        return href[
            join[
                'https://doi.org/',
                field('doi', raw=True)
            ],
            join[
                'doi:',
                field('doi', raw=True)
            ]
        ]
File:      ~/Documents/PhD/code/tlouf.github.io/.venv/lib/python3.9/site-packages/formatting/apa.py
Type:      method


In [4]:
from pybtex.plugin import find_plugin
from pybtex.database import parse_string
from pybtex.backends import markdown
APA = find_plugin('pybtex.style.formatting', 'apa')()
HTML = find_plugin('pybtex.backends', 'html')()
MD = markdown.Backend()
def bib2html(bibliography, exclude_fields=None):
    exclude_fields = exclude_fields or []
    if exclude_fields:
        bibliography = parse_string(bibliography.to_string('bibtex'), 'bibtex')
        for entry in bibliography.entries.values():
            for ef in exclude_fields:
                if ef in entry.fields.__dict__['_dict']:
                    del entry.fields.__dict__['_dict'][ef]
    formattedBib = APA.format_bibliography(bibliography)
    return [text_to_HTML(entry.text) for entry in formattedBib]

def text_to_HTML(text):
    return text.render(HTML).replace('<span class="bibtex-protected">', '').replace('</span>', '')
# def bib2md(bibliography, exclude_fields=None):
#     exclude_fields = exclude_fields or []
#     if exclude_fields:
#         bibliography = parse_string(bibliography.to_string('bibtex'), 'bibtex')
#         for entry in bibliography.entries.values():
#             for ef in exclude_fields:
#                 if ef in entry.fields.__dict__['_dict']:
#                     del entry.fields.__dict__['_dict'][ef]
#     formattedBib = APA.format_bibliography(bibliography)
#     return [entry.text.render(MD).replace('<span class="bibtex-protected">', '').replace('</span>', '') for entry in formattedBib]

In [11]:
parse_string(p.to_string('bibtex'), 'bibtex')

BibliographyData(
  entries=OrderedCaseInsensitiveDict([
    ('Louf2021', Entry('article',
      fields=[
        ('abstract', 'Cultural diversity encoded within languages of the world is at risk, as many languages have become endangered in the last decades in a context of growing globalization. To preserve this diversity, it is first necessary to understand what drives language extinction, and which mechanisms might enable coexistence. Here, we study language shift mechanisms using theoretical and data-driven perspectives. A large-scale empirical analysis of multilingual societies using Twitter and census data yields a wide diversity of spatial patterns of language coexistence. It ranges from a mixing of language speakers to segregation with multilinguals on the boundaries of disjoint linguistic domains. To understand how these different states can emerge and, especially, become stable, we propose a model in which language coexistence is reached when learning the other language is fac

In [13]:
pubs = pybtex.database.parse_file('me.bib')
my_name = 'Thomas Louf'
my_cit_name = 'Louf, T.'

for i, (bib_id, p) in enumerate(pubs.entries.items()):
    yaml_dict = {'collection': 'publications'}
    try:
        pub_year = p.fields['year']

        #todo: this hack for month and day needs some cleanup
        pub_month = p.fields.get('month')
        full_month = ''
        if pub_month is not None:
            if len(pub_month) < 3:
                pub_month = "0" + pub_month
                pub_month = pub_month[-2:]
            elif pub_month not in range(12):
                tmnth = time.strptime(pub_month[:3],'%b').tm_mon   
                pub_month = "{:02d}".format(tmnth) 
            else:
                pub_month = str(pub_month)
            full_month = time.strftime('%B', time.strptime(pub_month, '%m'))
        pub_day = p.fields.get('day')

        pub_date = '-'.join([x for x in [pub_year, pub_month, pub_day] if x is not None])
        if pub_month is None:
            pretty_date = pub_year
        elif pub_day is None:
            pretty_date = f'{full_month}, {pub_year}'
        else:
            pretty_date = f'{full_month} {pub_day}, {pub_year}'
        yaml_dict['print_date'] = pretty_date
        
        pub_title = p.fields['title']
        # strip out {} as needed (some bibtex entries that maintain formatting)
        clean_title = pub_title.replace("{", "").replace("}","").replace("\\","")
        yaml_dict['title'] = html_escape(clean_title)
        
        url_slug = re.sub("\\[.*\\]|[^a-zA-Z0-9_-]", "", clean_title.replace(" ","-")).replace("--","-")
        md_filename = (str(pub_date) + "-" + url_slug + ".md").replace("--","-")
        html_filename = (str(pub_date) + "-" + url_slug).replace("--","-")
        yaml_dict['permalink'] = '/publication/'  + html_filename
        
        # Build Citation from text
        citation = bib2html(p, exclude_fields=['month', 'url'])
        yaml_dict['citation'] = re.sub(r'"|[a-z&]{3,5};', '', html_escape(citation))
        yaml_dict['type'] = p.type
        # add venue logic depending on citation type
        if p.type != 'unpublished':
            if p.type == 'inproceedings':
                venue_key = 'booktitle'
            elif p.type == 'article':
                venue_key = 'journal'
            venue = p.fields.get(venue_key, '').replace("{", "").replace("}","").replace("\\","")
            yaml_dict['venue'] = html_escape(venue)
            
        authors = p.persons["author"]
        author_iter = zip(
            [' '.join(a.first_names) for a in authors],
            [' '.join(a.middle_names) for a in authors],
            [' '.join(a.last_names) for a in authors])
        authors = ', '.join([' '.join(a) for a in author_iter])
        authors = text_to_HTML(pybtex.richtext.Text.from_latex(authors))
        start_idx = authors.find(my_name)
        bolded = f'<b>{my_name}</b>'
        authors = authors[:start_idx] + bolded + authors[start_idx+len(my_name):]
        yaml_dict['authors'] = authors
        
        yaml_dict['paperurl'] = p.fields.get('url')
        yaml_dict['arxivId'] = p.fields.get('arxivId')
        # YAML variables
        md = '---\n'
        md += '\n'.join([f'{key}: "{value}"' for key, value in yaml_dict.items() if value])
        md += '\n---'
        # Content
        md += '\n' + text_to_HTML(pybtex.richtext.Text.from_latex(p.fields.get('abstract', '')))
        
        md_filename = os.path.basename(md_filename)
        with open("../_publications/" + md_filename, 'w') as f:
            f.write(md)
        print(f'SUCESSFULLY PARSED {bib_id}: \"', pub_title[:60],"..."*(len(pub_title)>60),"\"")
    # field may not exist for a reference
    except KeyError as e:
        print(f'WARNING Missing Expected Field {e} from entry {bib_id}: \"', pub_title[:30],"..."*(len(pub_title)>30),"\"")
        continue
    break

SUCESSFULLY PARSED Louf2021: " {Capturing the diversity of multilingual societies}  "
